In [1]:
import torch
import torch.nn as nn
from torch import Tensor
import time
import numpy as np
import random

In [2]:
SEED = 123
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
class CustomDropout(nn.Module):
    def __init__(self, p: float = 0.5, inplace: bool = False) -> None:
        super().__init__()
        if p < 0 or p > 1:
            raise ValueError(f"dropout probability has to be between 0 and 1, but got {p}")
        self.p = p
        self.inplace = inplace
        
    def forward(self, input: Tensor) -> Tensor:
        if self.training:
            mask = torch.bernoulli(torch.ones_like(input) * (1 - self.p)).to('cpu')
            return input * mask / (1 - self.p)
        return input

In [4]:
custom_dropout = CustomDropout(p=0.5).to('cpu')
torch_dropout = nn.Dropout(p=0.5).to('cpu')

In [5]:
x = torch.rand(5, 5).to('cpu')
x

tensor([[0.2961, 0.5166, 0.2517, 0.6886, 0.0740],
        [0.8665, 0.1366, 0.1025, 0.1841, 0.7264],
        [0.3153, 0.6871, 0.0756, 0.1966, 0.3164],
        [0.4017, 0.1186, 0.8274, 0.3821, 0.6605],
        [0.8536, 0.5932, 0.6367, 0.9826, 0.2745]])

In [6]:
custom_dropout.train()
torch_dropout.train()
output_custom_train = custom_dropout(x)
output_torch_train = torch_dropout(x)

In [7]:
custom_dropout.eval()
torch_dropout.eval()
output_custom_eval = custom_dropout(x)
output_torch_eval = torch_dropout(x)

In [8]:
num_runs = 1000

In [9]:
start_time_custom = time.time()
for _ in range(num_runs):
    custom_dropout(x)
end_time_custom = time.time()

start_time_torch = time.time()
for _ in range(num_runs):
    torch_dropout(x)
end_time_torch = time.time()

time_custom = end_time_custom - start_time_custom
time_torch = end_time_torch - start_time_torch

In [10]:
output_custom_train, output_torch_train

(tensor([[0.0000, 1.0331, 0.0000, 0.0000, 0.1479],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.6305, 1.3742, 0.1513, 0.3933, 0.0000],
         [0.8035, 0.2371, 1.6548, 0.0000, 1.3210],
         [0.0000, 1.1863, 0.0000, 0.0000, 0.0000]]),
 tensor([[0.5922, 0.0000, 0.5033, 0.0000, 0.1479],
         [0.0000, 0.2732, 0.2050, 0.3681, 1.4529],
         [0.0000, 1.3742, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 1.3210],
         [0.0000, 1.1863, 1.2735, 0.0000, 0.5490]]))

In [11]:
output_custom_eval, output_torch_eval

(tensor([[0.2961, 0.5166, 0.2517, 0.6886, 0.0740],
         [0.8665, 0.1366, 0.1025, 0.1841, 0.7264],
         [0.3153, 0.6871, 0.0756, 0.1966, 0.3164],
         [0.4017, 0.1186, 0.8274, 0.3821, 0.6605],
         [0.8536, 0.5932, 0.6367, 0.9826, 0.2745]]),
 tensor([[0.2961, 0.5166, 0.2517, 0.6886, 0.0740],
         [0.8665, 0.1366, 0.1025, 0.1841, 0.7264],
         [0.3153, 0.6871, 0.0756, 0.1966, 0.3164],
         [0.4017, 0.1186, 0.8274, 0.3821, 0.6605],
         [0.8536, 0.5932, 0.6367, 0.9826, 0.2745]]))

In [12]:
time_custom, time_torch

(0.0005321502685546875, 0.002566814422607422)